### Labic Images Segmentation - Main Notebook

#### Importando módulos

In [ ]:
import labic_init
labic_init.labic_init()

In [ ]:
from labic_images_segmentation import Dataset, DataAugmentation, SegmentationModel, SaveReport, PredictImages, DiceCoef

In [ ]:
import tensorflow as tf

#### Diretórios e parâmetros

In [ ]:
dataset_folder = "./TM40_Original"
norm_imgs_folder = "/Norm_images/*"
gt_folder = "/GT_images/*"

output_folder = "./TM40_46Prod"
test_imgs_folder = "/Producao/*"
gt_test_folder = "/GT_Producao/*"

ORIGINAL_SIZE = 512
NEW_SIZE = 256

batch_size = 4
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)

n_fold = 0
exec_folder_name = ""
title = "Linknet_resnet34"

#### Treinamento

##### Carregando dataset

In [ ]:
imgs = Dataset(folder=dataset_folder, norm_imgs_folder=norm_imgs_folder, gt_folder=gt_folder, ORIGINAL_SIZE=ORIGINAL_SIZE, NEW_SIZE=NEW_SIZE)

##### Dividindo dataset 

In [ ]:
imgs.split_dataset()

##### Data Augmentation

In [ ]:
data_aug = DataAugmentation(X_train=imgs.X_train, Y_train=imgs.Y_train, X_val=imgs.X_val, Y_val=imgs.Y_val, use_batch_size=batch_size)

##### Treinamento da rede

In [ ]:
segment = SegmentationModel(N=imgs.X_train.shape[-1], backbone_name="resnet34", trainDS=data_aug.trainDS, valDS=data_aug.valDS, epochs=epochs)

##### Salvando modelo

In [ ]:
save_report = SaveReport(model=segment.model, history=segment.history, folder_name=output_folder, n_fold=n_fold, use_batch_size=batch_size, epochs=epochs, exec_folder_name=exec_folder_name)

In [ ]:
save_report.save_history_txt()

#### Predição 

##### Carregando imagens de teste

In [ ]:
test_imgs = Dataset(folder=output_folder, norm_imgs_folder=test_imgs_folder, gt_folder=gt_test_folder, ORIGINAL_SIZE=ORIGINAL_SIZE, NEW_SIZE=NEW_SIZE)

#### Segmentando imagens de teste

In [ ]:
prediction = PredictImages(test_images=test_imgs, n_fold_folder_name=save_report.n_fold_folder_name, \
                           model_name=save_report.model_name, use_batch_size=batch_size, img_shape=test_imgs.img_shape)

##### Salvando DSC

In [ ]:
dice = DiceCoef(gt_imgs=test_imgs.Y, pred_folder=prediction.n_fold_folder_name+"/outputs_prod/*", new_size=NEW_SIZE)
dice.save_dice(f"{save_report.n_fold_folder_name}/dice_fold_{n_fold}.txt")

##### DSC final (n-folds)

In [ ]:
dice.generate_csv_dice(save_report=save_report, n_all_folders = n_fold, title=title)

##### Gráfico - Performance da Execução

In [ ]:
dice.generate_graphic(segment=segment, save_report=save_report, epochs=epochs)

##### DSC final (n-folds)

In [ ]:
dice.generate_csv_dice(save_report=save_report, n_all_folders = n_fold, title=title)

In [ ]:
dice.df